In [1]:
from recoxplainer.config import cfg
from recoxplainer.data_reader.data_reader import DataReader
from recoxplainer.models import GMFModel, ALS, EMFModel, BPR
from recoxplainer.recommender import Recommender
from recoxplainer.evaluator import Splitter, Evaluator
from recoxplainer.explain import KNNPostHocExplainer 
from recoxplainer.evaluator import ExplanationEvaluator
from recoxplainer.models.mlp_model import MLPModel
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
data = DataReader(**cfg.ml100k)
data.make_consecutive_ids_in_dataset()
data.binarize()
sp = Splitter()
train, test = sp.split_leave_n_out(data, frac=0.1)

In [3]:
als = ALS(**cfg.model.als)
als.fit(data)

/Users/mac/opt/anaconda3/envs/recoxplainer/lib/python3.6/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


  0%|          | 0/10 [00:00<?, ?it/s]

True

In [4]:
rec = Recommender(data, als)
recommendations = rec.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [5]:
recommendations.head

<bound method NDFrame.head of      userId  itemId  rank
161     0.0   166.0   1.0
85      0.0    89.0   2.0
387     0.0   404.0   3.0
7       0.0     8.0   4.0
240     0.0   247.0   5.0
..      ...     ...   ...
113   942.0   120.0   6.0
46    942.0    49.0   7.0
597   942.0   613.0   8.0
514   942.0   530.0   9.0
253   942.0   267.0  10.0

[9430 rows x 3 columns]>

In [6]:
KNNexplainer = KNNPostHocExplainer(als, recommendations, data)
KNNExpl = KNNexplainer.explain_recommendations()

Computing explanations:   0%|          | 0/9430 [00:00<?, ?it/s]

In [7]:
KNNExpl

,userId,itemId,rank,explanations
161,0.0,166.0,1.0,"{179, 423}"
85,0.0,89.0,2.0,"{665, 83, 86}"
387,0.0,404.0,3.0,"{289, 491, 751}"
7,0.0,8.0,4.0,"{528, 291, 179}"
240,0.0,247.0,5.0,{179}
...,...,...,...,...
113,942.0,120.0,6.0,"{24, 140}"
46,942.0,49.0,7.0,"{24, 209, 52}"
597,942.0,613.0,8.0,"{209, 140, 158}"
514,942.0,530.0,9.0,"{24, 140}"


In [8]:
ex = ExplanationEvaluator(data.num_user)

In [9]:
ex.model_fidelity(KNNExpl)

0.9917285259809111